In [71]:
!pip install -U langchain langchain-community langchain-core langchain-openai

In [72]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from google.colab import userdata
import os

In [73]:
try:
  openai_api_key = userdata.get('OPENAI_API_KEY')
  os.environ['OPENAI_API_KEY'] = openai_api_key
except KeyError:
  print("OPENAI_API_KEY not set")

In [74]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--task", default="Return a list of numbers")
# parser.add_argument("--language", default="rust")
# args = parser.parse_args()
task = "Return a function that returns a square of input number"
language = "python"
llm = OpenAI()

In [75]:
code_prompt = PromptTemplate(
    template="Write a short {language} program that will {task}",
    input_variables=["language", "task"]
    )

test_prompt = PromptTemplate(
    template="Write a test for the following {language} program:\n{code}",
    input_variables=["language", "code"]
    )

In [76]:
code_chain = LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code",
)

test_chain = LLMChain(
    llm=llm,
    prompt=test_prompt,
    output_key="test",
)

chain = SequentialChain(
    chains=[code_chain, test_chain],
    input_variables=["task", "language"],
    output_variables=["code", "test"],
)

result = chain({"task": task, "language": language})

In [77]:
print("Generated Code:\n", result["code"])

Generated Code:
 

def return_square():
    return lambda x: x**2
    
square = return_square()

print(square(5)) # Output: 25


In [78]:
print("Generated Test:\n", result["test"])

Generated Test:
 


import unittest

# Import the function to be tested
from return_square import return_square

class TestReturnSquare(unittest.TestCase):
    
    def test_return_square(self):
        # Test case 1: Input = 5, Expected output = 25
        self.assertEqual(return_square(5), 25, "Should return the square of the input")
        
        # Test case 2: Input = -3, Expected output = 9
        self.assertEqual(return_square(-3), 9, "Should return the square of the input")
        
        # Test case 3: Input = 0, Expected output = 0
        self.assertEqual(return_square(0), 0, "Should return the square of the input")
        
        # Test case 4: Input = 2.5, Expected output = 6.25
        self.assertEqual(return_square(2.5), 6.25, "Should return the square of the input")
        
if __name__ == '__main__':
    unittest.main()
